# A/B Testing for ShoeFly.com


In [4]:
import pandas as pd
ad_clicks = pd.read_csv('ad_clicks.csv')
display(ad_clicks.head())

,user_id,utm_source,day,ad_click_timestamp,experimental_group
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B


Your manager wants to know which ad platform is getting you the most views.

How many views (i.e., rows of the table) came from each utm_source?

In [9]:
ad_clicks.groupby("utm_source").user_id.count().reset_index()

,utm_source,user_id
0,email,255
1,facebook,504
2,google,680
3,twitter,215



If the column ad_click_timestamp is not null, then someone actually clicked on the ad that was displayed.

Create a new column called is_click, which is True if ad_click_timestamp is not null and False otherwise.

In [14]:
ad_clicks["is_click"]=~ad_clicks.ad_click_timestamp.isnull()
display(ad_clicks.head(5))

,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_click
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A,True
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B,False
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A,False
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B,False
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B,False


We want to know the percent of people who clicked on ads from each utm_source.

Start by grouping by utm_source and is_click and counting the number of user_id‘s in each of those groups. Save your answer to the variable clicks_by_source.

In [18]:
clicks_by_source=ad_clicks.groupby(['utm_source','is_click']).user_id.count().reset_index()
display(clicks_by_source)


,utm_source,is_click,user_id
0,email,False,175
1,email,True,80
2,facebook,False,324
3,facebook,True,180
4,google,False,441
5,google,True,239
6,twitter,False,149
7,twitter,True,66


Now let’s pivot the data so that the columns are is_click (either True or False), the index is utm_source, and the values are user_id.

Save your results to the variable clicks_pivot.

In [19]:
clicks_by_source.pivot(index="utm_source",values="user_id",columns="is_click").reset_index()

is_click,utm_source,False,True
0,email,175,80
1,facebook,324,180
2,google,441,239
3,twitter,149,66


## Analyzing an A/B Test
The column experimental_group tells us whether the user was shown Ad A or Ad B.

Were approximately the same number of people shown both adds?

In [20]:
ad_clicks.groupby('experimental_group').user_id.count().reset_index()

,experimental_group,user_id
0,A,827
1,B,827



Using the column is_click that we defined earlier, check to see if a greater percentage of users clicked on Ad A or Ad B.

In [31]:
clicks_by_exp_group=ad_clicks.groupby(['experimental_group',
                                       'is_click'])\
                                        .user_id.count().reset_index()

clicks_by_exp_group_pivot=clicks_by_exp_group.pivot(index="experimental_group",
                                                    values="user_id",
                                                    columns="is_click").reset_index()

display(clicks_by_exp_group_pivot)




is_click,experimental_group,False,True
0,A,517,310
1,B,572,255


The Product Manager for the A/B test thinks that the clicks might have changed by day of the week.

Start by creating two DataFrames: a_clicks and b_clicks, which contain only the results for A group and B group, respectively.

In [33]:
a_clicks=ad_clicks[ad_clicks["experimental_group"]=="A"]
b_clicks=ad_clicks[ad_clicks["experimental_group"]=="B"]

For each group (a_clicks and b_clicks), calculate the percent of users who clicked on the ad by day.

In [37]:
a_clicks.groupby(["is_click","day"]).user_id.count().reset_index()\
.pivot(index="is_click",values="user_id",columns="day").reset_index()

day,is_click,1 - Monday,2 - Tuesday,3 - Wednesday,4 - Thursday,5 - Friday,6 - Saturday,7 - Sunday
0,False,70,76,86,69,77,73,66
1,True,43,43,38,47,51,45,43


In [38]:
b_clicks.groupby(["is_click","day"]).user_id.count().reset_index()\
.pivot(index="is_click",values="user_id",columns="day").reset_index()

day,is_click,1 - Monday,2 - Tuesday,3 - Wednesday,4 - Thursday,5 - Friday,6 - Saturday,7 - Sunday
0,False,81,74,89,87,90,76,75
1,True,32,45,35,29,38,42,34
